<a href="https://colab.research.google.com/github/jpbeaud/language/blob/main/essai1_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install tensorflow
!pip install keras-preprocessing

Création & préparation du modèle

In [11]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Définition des paramètres
num_encoder_tokens = 100
num_decoder_tokens = 100
latent_dim = 256

# Encodeur
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]
print(encoder_inputs)
# Décodeur
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Modèle Seq2Seq
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()


<KerasTensor shape=(None, None, 100), dtype=float32, sparse=False, name=keras_tensor_9>


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, None, 100)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, None, 100)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 256), (None,   │        365,568 │ input_layer_2[0][0]    │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, None, 256),    │        365,568 │ input_layer_3[0][0],   │
│                           │ (None, 256), (None,    │                │ lstm_2[0][1],          │
│                           │ 256)]                  │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, None, 100)      │         25,700 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 756,836 (2.89 MB)

 Trainable params: 756,836 (2.89 MB)

 Non-trainable params: 0 (0.00 B)

Préparation des données

In [12]:

from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import numpy as np



# Exemples de données (à remplacer par tes propres données)
input_texts = ["bonjour", "salut"]
target_texts = ["hello", "hi"]

# Tokenisation des textes
input_tokenizer = Tokenizer(num_words=num_encoder_tokens)
print("num_encorder_tokens = ", num_encoder_tokens)
print("num_decoder_tokens = ", num_decoder_tokens)
input_tokenizer.fit_on_texts(input_texts)
encoder_input_data = input_tokenizer.texts_to_sequences(input_texts)
encoder_input_data = pad_sequences(encoder_input_data, padding='post')

target_tokenizer = Tokenizer(num_words=num_decoder_tokens)
target_tokenizer.fit_on_texts(target_texts)
decoder_input_data = target_tokenizer.texts_to_sequences(target_texts)
decoder_input_data = pad_sequences(decoder_input_data, padding='post')


num_encorder_tokens =  100
num_decoder_tokens =  100


Entrainement du modèle avec les données préparées

In [22]:
# Convertir les données cibles en une représentation à une dimension
decoder_target_data = np.zeros((len(target_texts), max([len(t) for t in decoder_input_data]), num_decoder_tokens))
for i, seqs in enumerate(decoder_input_data):
    #print(i, seqs)
    for t, token in enumerate(seqs):
        #print(t, token)
        if token.all() > 0:
            decoder_target_data[i, t, token] = 1.0
# Convertir les données sources en une représentation à une dimension
encoder_input_data = np.zeros((len(input_texts), max([len(t) for t in encoder_input_data]), num_encoder_tokens))
for i, seqs in enumerate(encoder_input_data):
    #print(i, seqs)
    for t, token in enumerate(seqs):
        #print(t, token)
        if token.all() > 0:
            encoder_input_data[i, t, token] = 1.0
decoder_input_data = np.zeros((len(input_texts), max([len(t) for t in decoder_input_data]), num_decoder_tokens))
for i, seqs in enumerate(decoder_input_data):
    #print(i, seqs)
    for t, token in enumerate(seqs):
        #print(t, token)
        if token.all() > 0:
            decoder_input_data[i, t, token] = 1.0

# Entraîner le modèle
print(type(encoder_input_data))
print(decoder_input_data.shape)
print(decoder_target_data.shape)
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=64,epochs=100,validation_split=0.2)


<class 'numpy.ndarray'>
(2, 1, 100)
(2, 1, 100)
Epoch 1/100


AttributeError: 'NoneType' object has no attribute 'items'